In [1]:
from qiskit import Aer, QuantumCircuit, transpile, assemble
from qiskit.visualization import plot_histogram
import numpy as np

In [2]:
# Generate random expected returns for assets
np.random.seed(42)
num_assets = 3
expected_returns = np.random.rand(num_assets)